In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: shawn
"""
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

## Generate MKT, SMB and HML factor

In [3]:
df = pd.read_csv('/Users/shawn/Github/M1/金融計量/TWstocks.csv')
df.head()

,證券代碼,年月,收盤價(元)_月,報酬率％_月,流通在外股數(千股),市值(百萬元),市值比重％,開盤價(元)_月,最高價(元)_月,最低價(元)_月,...,本益比-TEJ,股價淨值比-TSE,股價淨值比-TEJ,股價營收比-TEJ,股利殖利率-TSE,現金股利率,股價漲跌(元),高低價差%,市場別,該月結束日
0,1101 台泥,201001,10.92,-12.2058,3292175,98271,0.516,12.43,13.31,10.68,...,16.14,1.28,1.28,4.57,4.42,4.42,-4.15,21.1765,TSE,20100129
1,1102 亞泥,201001,12.76,-12.4276,2985735,90468,0.475,14.57,15.20,12.57,...,12.67,1.26,1.26,8.56,6.93,5.77,-4.30,18.0636,TSE,20100129
2,1103 嘉泥,201001,9.23,-12.7072,717877,11342,0.060,10.63,11.04,9.06,...,56.97,0.68,0.68,4.50,0.00,0.00,-2.30,18.7845,TSE,20100129
3,1104 環泥,201001,7.54,-7.1428,603891,9813,0.052,8.12,9.10,7.31,...,65.70,0.81,0.81,3.16,0.00,0.00,-1.25,22.0000,TSE,20100129
4,1108 幸福,201001,5.03,-11.5084,404738,3206,0.017,5.68,5.94,4.96,...,NaN,0.75,0.75,1.09,0.00,0.00,-1.03,17.3184,TSE,20100129


## SMB

In [4]:
df['帳面市值比'] = 1/df['股價淨值比-TEJ']
ME = df.groupby('年月')['市值(百萬元)'].apply(lambda x: x.median())
ME.name = '市值_中位數'
df = df.merge(ME, on='年月')
df['市值matrix'] = np.where(df['市值(百萬元)']>df['市值_中位數'], 'B', 'S')


In [5]:
df1 = (df.groupby(['年月','市值matrix'])['市值(百萬元)'].sum()).reset_index()
df = df.merge(df1, on=['年月','市值matrix'])
df['weight'] = df['市值(百萬元)_x']/df['市值(百萬元)_y']
df.groupby(['年月','市值matrix'])['weight'].sum()

df.groupby(['年月','市值matrix'])['weight'].sum()

年月      市值matrix
201001  B           1.0
        S           1.0
201002  B           1.0
        S           1.0
201003  B           1.0
                   ... 
202310  S           1.0
202311  B           1.0
        S           1.0
202312  B           1.0
        S           1.0
Name: weight, Length: 336, dtype: float64

In [6]:
df.head()

,證券代碼,年月,收盤價(元)_月,報酬率％_月,流通在外股數(千股),市值(百萬元)_x,市值比重％,開盤價(元)_月,最高價(元)_月,最低價(元)_月,...,現金股利率,股價漲跌(元),高低價差%,市場別,該月結束日,帳面市值比,市值_中位數,市值matrix,市值(百萬元)_y,weight
0,1101 台泥,201001,10.92,-12.2058,3292175,98271,0.516,12.43,13.31,10.68,...,4.42,-4.15,21.1765,TSE,20100129,0.781250,5694.0,B,17461040,0.005628
1,1102 亞泥,201001,12.76,-12.4276,2985735,90468,0.475,14.57,15.20,12.57,...,5.77,-4.30,18.0636,TSE,20100129,0.793651,5694.0,B,17461040,0.005181
2,1103 嘉泥,201001,9.23,-12.7072,717877,11342,0.060,10.63,11.04,9.06,...,0.00,-2.30,18.7845,TSE,20100129,1.470588,5694.0,B,17461040,0.000650
3,1104 環泥,201001,7.54,-7.1428,603891,9813,0.052,8.12,9.10,7.31,...,0.00,-1.25,22.0000,TSE,20100129,1.234568,5694.0,B,17461040,0.000562
4,1110 東泥,201001,9.68,-3.0837,572000,6292,0.033,10.03,11.53,9.51,...,0.00,-0.35,20.2643,TSE,20100129,1.333333,5694.0,B,17461040,0.000360


In [7]:
df['return1'] = df['報酬率％_月']* df['weight']
SMB = df.groupby(['年月','市值matrix'])['return1'].sum()


In [8]:
SMB
SMB = SMB.to_frame().reset_index()
SMB.set_index('年月',drop=True, inplace=True)

In [9]:
SMB = SMB[SMB['市值matrix']=='S']['return1'] - SMB[SMB['市值matrix']=='B']['return1']
SMB.name = 'SMB'
SMB

年月
201001   -0.817088
201002    0.859611
201003    5.837776
201004    1.061575
201005   -1.252764
            ...   
202308   -0.096528
202309    1.809980
202310   -0.046480
202311   -0.033274
202312   -1.116880
Name: SMB, Length: 168, dtype: float64

## HML

In [10]:
a = df.groupby('年月')['帳面市值比'].quantile(0.7)
a.name = 'BM_0.7'
b = df.groupby('年月')['帳面市值比'].quantile(0.3)
b.name = 'BM_0.3'
df = df.merge(a, on='年月')
df = df.merge(b, on='年月')
df['BM_matrix'] = np.where(df['帳面市值比']>df['BM_0.7'], 'V', (np.where(df['帳面市值比']<df['BM_0.3'],'G', 'N')))

In [11]:
df2 = (df.groupby(['年月','BM_matrix'])['市值(百萬元)_x'].sum()).reset_index()
df = df.merge(df2, on=['年月','BM_matrix'])
df['weight2'] = df['市值(百萬元)_x_x']/df['市值(百萬元)_x_y']
df.groupby(['年月','BM_matrix'])['weight2'].sum()

年月      BM_matrix
201001  G            1.0
        N            1.0
        V            1.0
201002  G            1.0
        N            1.0
                    ... 
202311  N            1.0
        V            1.0
202312  G            1.0
        N            1.0
        V            1.0
Name: weight2, Length: 504, dtype: float64

In [12]:
df['return2'] = df['報酬率％_月']* df['weight2']
HML = df.groupby(['年月','BM_matrix'])['return2'].sum()
HML = HML.to_frame().reset_index()
HML.set_index('年月',drop=True, inplace=True)
HML = HML[HML['BM_matrix']=='V']['return2'] - HML[HML['BM_matrix']=='G']['return2']
HML.name = 'HML'
HML

年月
201001   -1.246261
201002   -1.260319
201003    0.732844
201004   -1.857605
201005   -2.703659
            ...   
202308   -0.952760
202309    1.354797
202310   -2.501113
202311   -4.216089
202312    1.139341
Name: HML, Length: 168, dtype: float64

In [13]:
fama = pd.concat([SMB,HML],axis=1)
fama

,SMB,HML
年月,,
201001,-0.817088,-1.246261
201002,0.859611,-1.260319
201003,5.837776,0.732844
201004,1.061575,-1.857605
201005,-1.252764,-2.703659
...,...,...
202308,-0.096528,-0.952760
202309,1.809980,1.354797
202310,-0.046480,-2.501113


## MKT

In [14]:
mkt = pd.read_csv('/Users/shawn/Github/M1/金融計量/TWII.csv')
mkt.head()

,證券代碼,年,月,報酬率％_月
0,Y9999 加權指數,2010,1,-6.6886
1,Y9999 加權指數,2010,2,-2.6745
2,Y9999 加權指數,2010,3,6.5083
3,Y9999 加權指數,2010,4,1.0630
4,Y9999 加權指數,2010,5,-7.8742


In [15]:
rm = mkt['報酬率％_月']
rm

0     -6.6886
1     -2.6745
2      6.5083
3      1.0630
4     -7.8742
        ...  
163   -2.9801
164   -1.6878
165   -2.1554
166    8.9530
167    1.8236
Name: 報酬率％_月, Length: 168, dtype: float64

## Merge all factors

In [16]:
fama['rm'] = rm.values
fama
fama = fama.rename_axis('date')

In [17]:
fama.to_csv('/Users/shawn/Github/M1/金融計量/ff3.csv')

In [18]:
fama

,SMB,HML,rm
date,,,
201001,-0.817088,-1.246261,-6.6886
201002,0.859611,-1.260319,-2.6745
201003,5.837776,0.732844,6.5083
201004,1.061575,-1.857605,1.0630
201005,-1.252764,-2.703659,-7.8742
...,...,...,...
202308,-0.096528,-0.952760,-2.9801
202309,1.809980,1.354797,-1.6878
202310,-0.046480,-2.501113,-2.1554
